Перед нами стоит задача предсказания числового значения на основе заданного текста. По заголовку мы должны предсказать его числовой рейтинг от 0.0 до 10.0 с точностью 0.1. 

В качестве библиотеки, реализующей модель русского языка, мы используем spaCy (в первую очередь из соображений скорости).

- Лемматизация
- Составление триграмм
- Категоризация отрезков текста

In [2]:
nlp = spacy.load("ru_core_news_lg")

In [1]:
import pickle
import numpy as np
import pandas as pd

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# https://spacy.io/models/ru
import spacy
# spacy.prefer_gpu()
nlp = spacy.load("ru_core_news_sm")

from tqdm.auto import tqdm
tqdm.pandas()

DATASETS_PATH = '/home/leo/DATASETS'

# with open('../sources.pickle', 'rb') as f:
#     sources = pickle.load(f)
    
# dfs = dict()
# for source in sources:
#     dfs[source] = pd.read_csv(f"{DATASETS_PATH}/{source}.csv", index_col=0, parse_dates=['post_time', 'parse_time'])
#     dfs[source]['source'] = source

# df = pd.concat(dfs[key] for key in dfs)
    
# # удаляем дубликаты
# df = df.drop_duplicates()    

# df = pd.concat([dfs[key] for key in dfs])
# df.views_num = df.views_num.apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))
# df.views_num = np.log(df.views_num)/np.log(df.views_num.max())

# # удаляем закрытые записи без просмотров
# df = df.drop(df[df.views_num == np.NINF].index)

# Оценка заголовка
Есть две модели: модель генерации заголовка и модель оценки заголовка. Начнем с модели оценки.

In [3]:
with open(f"{DATASETS_PATH}/title.pickle", 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    Xy = pickle.load(f)

In [ ]:
# Xy

In [ ]:
# Xy = df[['title', 'views_num']]

# # генерация дополнительных признаков
# Xy.loc[:, ['title']] = Xy.title.apply(str)

# Xy.loc[:, ['doc']] = Xy.title.progress_apply(nlp)

# # длина заголовка в символах
# Xy.loc[:, ['len']] = Xy.title.apply(len)

In [ ]:
# # количество токенов
# Xy.loc[:, ['tokens_num']] = Xy.tokens.apply(lambda x: len(x))

In [4]:
Xy['doc'] = Xy['tokens']

Начнем с базовой модели: сопоставим каждому заголовку векторное представление, обучим модель для оценки заголовков и проверим качество скользящим контролем.

In [5]:
Xy.loc[:, ['vector']] = Xy.doc.apply(lambda x: x.vector)

In [6]:
X = Xy['vector']

In [7]:
y = Xy['views_num']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
X_train= np.stack(X_train.values)

In [10]:
X_train = torch.from_numpy(X_train).float().to(device)

In [11]:
y_train = torch.from_numpy(y_train.values).float().to(device)

In [12]:
model = torch.nn.Sequential(
        torch.nn.Linear(96, 200),
        torch.nn.ReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 1)
        )

In [13]:
model.to(device)

Sequential(
  (0): Linear(in_features=96, out_features=200, bias=True)
  (1): ReLU()
  (2): Linear(in_features=200, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=50, bias=True)
  (5): ReLU()
  (6): Linear(in_features=50, out_features=1, bias=True)
)

In [14]:
criterion = torch.nn.MSELoss(reduction='sum')

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [24]:
torch.cuda.empty_cache()

In [16]:
for epoch in tqdm(range(10)):
    y_pred = model(X_train.requires_grad_())
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print('Epoch: ', epoch, 'Loss: ', loss.item())

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([180436])) that is different to the input size (torch.Size([180436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: CUDA out of memory. Tried to allocate 121.29 GiB (GPU 0; 1.96 GiB total capacity; 550.12 MiB already allocated; 535.62 MiB free; 558.00 MiB reserved in total by PyTorch)

In [23]:
import gc
gc.collect()

149

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
# Xy.sort_values(by='tokens_num', ascending=False)

In [ ]:
with open(f"{DATASETS_PATH}/title.pickle", 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(Xy, f, pickle.HIGHEST_PROTOCOL)

Для проверки модели используем [кросс-валидацию](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9A%D1%80%D0%BE%D1%81%D1%81-%D0%B2%D0%B0%D0%BB%D0%B8%D0%B4%D0%B0%D1%86%D0%B8%D1%8F) по 5 фолдам.

In [ ]:
# def load_text(source, filename):
#     try:
#         with open(f'{DATASETS_PATH}/{source}/{filename}', 'r') as f:
#             return f.read()
#     except FileNotFoundError:
#         return ''
    
# df['full_text'] = df.apply(lambda x: load_text(x.source, x.filename), axis=1)